### <b style="color: #abcdef">03.이미지 비전</b>

> #### <b style="color: #58a491">객체검출</b>
> - 컨볼루션 레이어는 위치정보를 추출함
> - 컨볼루션 스택에 요소추가하여 네트워크 훈련

> #### <b style="color: #58a491">YOLO</b>
> - 컨볼루션 백본 끝에 요소 추가
> - YOLO 격자
> - 중심좌표, 너비, 높이, 확신도 => (x, y, w, h, C)